# CrewAI

In [ ]:
# pip install openai
# !pip install --upgrade crewai
# !pip install crewai crewai-tools

In [ ]:
import openai
import os
import getpass

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
budgeting_agent = Agent(
    role="Budgeting Advisor",
    goal="Create a monthly budget to help users manage their income and expenses effectively.",
    backstory=(
        "You are an experienced financial advisor specializing in personal finance. "
        "Your goal is to help users allocate their income efficiently, ensuring they cover "
        "all necessary expenses while also saving for the future."
    ),
    allow_delegation=False,
    verbose=True
)


In [ ]:
investment_agent = Agent(
    role="Investment Advisor",
    goal="Recommend suitable investment options based on the user's financial goals and risk tolerance.",
    backstory=(
        "You are an investment expert with years of experience in the financial markets. "
        "Your goal is to help users grow their wealth by providing sound investment advice "
        "tailored to their risk tolerance and financial objectives."
    ),
    allow_delegation=False,
    verbose=True
)


In [ ]:
debt_management_agent = Agent(
    role="Debt Management Specialist",
    goal="Help users manage and reduce their debt through effective strategies.",
    backstory=(
        "You specialize in helping individuals overcome debt by creating personalized repayment plans. "
        "Your focus is on reducing interest payments and improving the user's financial health."
    ),
    allow_delegation=False,
    verbose=True
)


In [ ]:
budgeting_task = Task(
    description=(
        "1. Analyze the user's income and expenses.\n"
        "2. Create a detailed monthly budget that includes essential expenses, savings, and discretionary spending.\n"
        "3. Provide tips for optimizing spending and increasing savings."
    ),
    expected_output="A comprehensive monthly budget with recommendations for optimizing spending and savings.",
    agent=budgeting_agent
)


In [ ]:
investment_task = Task(
    description=(
        "1. Assess the user's financial goals and risk tolerance.\n"
        "2. Recommend suitable investment options such as stocks, bonds, mutual funds, or ETFs.\n"
        "3. Provide a brief overview of each recommended investment's potential risks and returns."
    ),
    expected_output="A personalized investment plan with recommendations and risk assessments.",
    agent=investment_agent
)


In [ ]:
debt_management_task = Task(
    description=(
        "1. Analyze the user's current debts, including interest rates and balances.\n"
        "2. Develop a repayment plan that prioritizes high-interest debt and suggests strategies for paying off balances faster.\n"
        "3. Provide advice on consolidating debt or negotiating lower interest rates."
    ),
    expected_output="A debt management plan with actionable steps to reduce and eliminate debt.",
    agent=debt_management_agent
)


In [ ]:
crew = Crew(
    agents=[budgeting_agent, investment_agent, debt_management_agent],
    tasks=[budgeting_task, investment_task, debt_management_task],
    verbose=True  # Set to True for detailed logging or False to reduce output
)



In [ ]:
user_financial_data = {
    "income": 5000,  # Monthly income in dollars
    "expenses": {
        "rent": 1500,
        "utilities": 300,
        "groceries": 400,
        "transportation": 200,
        "entertainment": 150,
        "other": 450
    },
    "debts": {
        "credit_card": {
            "balance": 2000,
            "interest_rate": 0.18  # 18% interest rate
        },
        "student_loan": {
            "balance": 15000,
            "interest_rate": 0.045  # 4.5% interest rate
        }
    },
    "savings_goal": 500  # Monthly savings goal in dollars
}

# Now run the crew kickoff with the defined data
result = crew.kickoff(inputs=user_financial_data)

# Extract the raw text from the result
raw_result = result.raw

# Display the result as markdown
from IPython.display import Markdown
Markdown(raw_result)

 




# New Product Launch

In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from pydantic import BaseModel

In [ ]:
os.environ["SERPER_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [ ]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [ ]:
market_researcher = Agent(
    role="Market Researcher",
    goal="Conduct thorough market research to identify target demographics and competitors.",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Analytical and detail-oriented, you excel at gathering insights about the market, "
        "analyzing competitors, and identifying the best strategies to target the desired audience."
    )
)


In [ ]:
content_creator = Agent(
    role='Content Creator',
    goal="Develop engaging content for the product launch, including blogs, social media posts, and videos.",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and persuasive, you craft content that resonates with the audience, "
        "driving engagement and excitement for the product launch."
    )
)


In [ ]:
pr_outreach_specialist = Agent(
    role="PR and Outreach Specialist",
    goal="Reach out to influencers, media outlets, and key opinion leaders to promote the product launch.",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With strong networking skills, you connect with influencers and media outlets to ensure "
        "the product launch gains maximum visibility and coverage."
    )
)


In [ ]:
class MarketResearchData(BaseModel):
    target_demographics: str
    competitor_analysis: str
    key_findings: str


In [ ]:
market_research_task = Task(
    description="Conduct market research for the {product_name} launch, focusing on target demographics and competitors.",
    expected_output="A detailed report on market research findings, including target demographics and competitor analysis.",
    human_input=True,
    output_json=MarketResearchData,
    output_file="market_research.json",
    agent=market_researcher
)


In [ ]:
content_creation_task = Task(
    description="Create content for the {product_name} launch, including blog posts, social media updates, and promotional videos.",
    expected_output="A collection of content pieces ready for publication.",
    human_input=True,
    async_execution=False,  # Change to synchronous
    output_file="content_plan.txt",
    agent=content_creator
)



In [ ]:
pr_outreach_task = Task(
    description="Contact influencers, media outlets, and key opinion leaders to promote the {product_name} launch.",
    expected_output="A report on outreach efforts, including responses from influencers and media coverage.",
    async_execution=False,  # Change to synchronous
    output_file="outreach_report.md",
    agent=pr_outreach_specialist
)



In [ ]:
product_launch_crew = Crew(
    agents=[market_researcher, content_creator, pr_outreach_specialist],
    tasks=[market_research_task, content_creation_task, pr_outreach_task],  # Ensure only one async task is at the end
    verbose=True
)


In [ ]:
launch_details = {
    'product_name': "SmartHome 360",
    'product_description': "A cutting-edge smart home system that integrates with all your devices.",
    'launch_date': "2024-10-01",
    'target_market': "Tech-savvy homeowners",
    'budget': 50000
}

result = product_launch_crew.kickoff(inputs=launch_details)


In [ ]:
import json 
from pprint import pprint  
from IPython.display import Markdown  # Add this line to import the Markdown function

# Display the generated market_research.json file
with open('market_research.json') as f:
   data = json.load(f)
pprint(data)

# Display the generated content_plan.txt file
with open('content_plan.txt') as f:
    content = f.read()
print(content)

# Display the generated outreach_report.md file
Markdown("outreach_report.md")


# RAG

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import WebsiteSearchTool

# Step 1: Configure the WebsiteSearchTool with the specific website URL
search_tool = WebsiteSearchTool(website='https://en.wikipedia.org/wiki/Alan_Turing')

# Step 2: Create an Agent who will perform the search
search_agent = Agent(
    role='Website Researcher',
    goal='Search and extract relevant information from a specific website.',
    verbose=True,
    memory=True,
    backstory='You are an expert in searching websites for the most relevant and up-to-date information.',
    tools=[search_tool]
)

# Step 3: Define a Task for the Agent
search_task = Task(
    description=(
        "Use the provided website to find information on the topic '{topic}'. "
        "Make sure to gather all the relevant data available on this site."
    ),
    expected_output='A detailed summary of the information found on the website.',
    tools=[search_tool],
    agent=search_agent,
)

# Step 4: Create a Crew with the agent and task
research_crew = Crew(
    agents=[search_agent],
    tasks=[search_task],
    process=Process.sequential  # Executes tasks one after the other
)

# Step 5: Kickoff the crew with specific input
result = research_crew.kickoff(inputs={'topic': 'Artificial intelligence'})

# Print the results
print(result)
